In [ ]:
dataset_path = os.path.join(os.getcwd(), "MNIST")

# TEST

In [ ]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [ ]:
images_train, label_train = load_mnist(dataset_path)
images_test, label_test = load_mnist(dataset_path, kind= 't10k')

### Categorie
```
0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot
```

In [11]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

import torch
# import torch.nn as nn
# from torch.autograd import Variable

# import torchvision
# import torchvision.transforms as transforms
# from torch.utils.data import Dataset, DataLoader
# from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'torch'

In [ ]:
train_set = load_mnist(dataset_path)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(images_train.shape)
x= images_train.reshape(60000, 28,28)
print(x.shape)
print(images_train)


In [ ]:
index=0 # change this number 
plt.imshow(x[index], cmap='gray') # printing 10th image
plt.colorbar() # shows the bar on the right side of the image
plt.grid(True) # will shot the grid
plt.show()

Domande da porsi:
- Nel Dataloader vanno inseriti i dati grezzi o con il reshape fatto?
- le label e le immagini dedovo essere messi all'interno dello stesso tensore? In base all esempio che fa il prof i dati sono nello stesso tensore

In [ ]:
random_seed = 1
torch.manual_seed(random_seed)

tf=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(  (0.1307,), (0.3081,) )
                    ])

tf_noNorm=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor()
                     ])

In [ ]:
xttrain= tf_noNorm(images_train)
xttest= tf_noNorm(images_test)

In [ ]:
print(x2, ds_train)

In [ ]:
batch_size_train = 64
train_loader = DataLoader(xttest, batch_size=batch_size_train)


batch_size_test = 1000
test_loader = DataLoader(xttest, batch_size=batch_size_test, shuffle=True)

In [ ]:
# carica tutto il dataset 
examples = enumerate(train_loader)
batch_idx, example_data = next(examples)

example_data= example_data.reshape(10000, 1, 28, 28)
#Il formato e' (Numero_campioni, canali, dimensione_verticale, dimensione_orizzontale)


In [ ]:
f, axs = plt.subplots(3,3)
plt.tight_layout()
k = 0
for i in range(3):
    for j in range(3):
       
        axs[i][j].imshow(example_data[k][0], cmap='gray', interpolation='none')
        axs[i][j].set_title("Ground Truth: {}".format(label_train[k]))
        #axs[i][j].set_xticks([])
        #axs[i][j].set_yticks([])
        k += 1
plt.show()

Architettura della rete
La rete deve prendere in ingresso il vettore rappresentativo delle immagini e dare in output la classe della immagine, quindi:

input
28×28 

output
10 
una uscita per ogni classe

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
# ottimizzatore
import torch.optim as optim

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        # x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
# Definiamo la Rete e l'ottimizzatore
learning_rate = 0.01
momentum = 0.5 

network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
print(network)

In [ ]:
n_epochs = 3

log_interval = 50  # passo a cui salvare i dati di log

In [ ]:
# definizione della routine di training 
# la routine esegue una eopca di training
#La loss usata e' la negative log likelihood ```loss = F.nll_loss(output, target)``` che ha un senso perche' la uscita e' calcolata come ```F.log_softmax(x)``` e quindi e' interpretata come log-probability.
def train(train_loader, epoch, bs_train, net, opt, dir_results, tr_ls, tr_ct):
    net.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        # si azzerao i gradienti
        opt.zero_grad()
        # si calcola l'output della rete
        output = net(data)
        # si calcola l'errore con la funzione di loss
        loss = F.nll_loss(output, target)
        # si calcolano i gradienti
        loss.backward()
        # l'ottimizzaotre segue il passo di aprendimento
        opt.step()

        # print e report.........................
        if batch_idx % log_interval == 0:

            num_campioni_elaborati = batch_idx * len(data)
            num_campioni_totali = len(train_loader.dataset)
                        
            tr_ls.append(loss.item())

            totale_campioni_elaborati =  (batch_idx*bs_train) + (epoch-1)*num_campioni_totali 
        
            tr_ct.append( totale_campioni_elaborati )

            print('Train Epoch: {} [{}/{}  ({:.0f}%)]  \tLoss: {:.6f}'.format(
                epoch, num_campioni_elaborati, num_campioni_totali , 
                100. * batch_idx / len(train_loader), 
                loss.item()
                )
            )

            # salvataggio rete e ottimizzatore
            torch.save(network.state_dict(), dir_results + 'model.pth')
            torch.save(optimizer.state_dict(), dir_results + 'optimizer.pth')
        # fine print e report.........................    
         
    return tr_ls, tr_ct

In [ ]:
# esegue un passo di test
def test(tls, net, test_ld):

    net.eval()
    test_loss = 0
    correct = 0

    # evita che si calcolino i gradienti
    with torch.no_grad():
        # visto che i dati di test sono 10.000 e il batch_size_tset e' 
        # 1000 la valutazione e' divisa in 10 passi 
        for data, target in test_ld:
            output = net(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()

    test_loss /= len(test_ld.dataset)
    tls.append(test_loss)

    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_ld.dataset),
        100. * correct / len(test_ld.dataset)))
    return tls

In [ ]:
##STAND BY capire come trasformare il dataset

labels = {0 : "T-shirt/top", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat",
          5: "Sandal", 6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle Boot"}

IMG_ROWS,IMG_COLS = 28,28
NUM_CLASSES = 10
TEST_SIZE = 0.2
RANDOM_STATE = 2018
#Model
NO_EPOCHS = 50
BATCH_SIZE = 128

def sample_images_data(data):
    # An empty list to collect some samples
    sample_images = []
    sample_labels = []

    # Iterate over the keys of the labels dictionary defined in the above cell
    for k in labels.keys():
        # Get four samples for each category
        samples = data[data["label"] == k].head(4)
        # Append the samples to the samples list
        for j, s in enumerate(samples.values):
            # First column contain labels, hence index should start from 1
            img = np.array(samples.iloc[j, 1:]).reshape(IMG_ROWS,IMG_COLS)
            sample_images.append(img)
            sample_labels.append(samples.iloc[j, 0])

    print("Total number of sample images to plot: ", len(sample_images))
    return sample_images, sample_labels

In [ ]:
##STAND BY capire come trasformare il dataset

def plot_sample_images(data_sample_images,data_sample_labels,cmap="Grey"):
    # Plot the sample images now
    f, ax = plt.subplots(5,8, figsize=(16,10))

    for i, img in enumerate(data_sample_images):
        ax[i//8, i%8].imshow(img, cmap=cmap)
        ax[i//8, i%8].axis('off')
        ax[i//8, i%8].set_title(labels[data_sample_labels[i]])
    plt.show()    
    
train_sample_images, train_sample_labels = sample_images_data()
plot_sample_images(train_sample_images,train_sample_labels, "Greens")